In [1]:
import pandas as pd
from quantrl_lab.data import (
    DataSourceRegistry,
    DataProcessor,
    IndicatorRegistry,
    AlpacaDataLoader, 
    YfinanceDataloader, 
    AlphaVantageDataLoader
)
from quantrl_lab.data.indicators.technical_indicators import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

#### What each data source supports (as of the latest implementation)

In [2]:
data_source_apc = AlpacaDataLoader()
supported_features = data_source_apc.supported_features
print(f"Supported features for AlpacaDataLoader: {supported_features}") 

data_source_yf = YfinanceDataloader()
supported_features = data_source_yf.supported_features
print(f"Supported features for YfinanceDataLoader: {supported_features}")

data_source_av = AlphaVantageDataLoader()
supported_features = data_source_av.supported_features
print(f"Supported features for AlphaVantageDataLoader: {supported_features}")

Supported features for AlpacaDataLoader: ['historical_bars', 'news', 'live_data', 'streaming']
Supported features for YfinanceDataLoader: ['historical_bars']
Supported features for AlphaVantageDataLoader: ['historical_bars', 'news']


#### The technical indicators that are supported as of now (more can be added later on)

In [3]:
print(f"Available indicators:  {IndicatorRegistry.list_all()}")

Available indicators:  ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']


In [4]:
data_loader = DataSourceRegistry()

In [5]:
olhcv_df = data_loader.get_historical_ohlcv_data(
    symbols="MU",
    start="2023-01-01",
    end="2025-06-30",
    timeframe="1d",
)

2025-07-09 23:58:18.078 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_historical_ohlcv_data:156 - Fetching historical data for MU from 2023-01-01 to 2025-06-30 with timeframe 1d


In [6]:
olhcv_df.head()

,Symbol,Timestamp,Open,High,Low,Close,Volume,Trade_count,VWAP,Date
0,MU,2023-01-03 05:00:00+00:00,50.56,50.960,49.55,50.37,14357184.0,126221.0,50.323799,2023-01-03
1,MU,2023-01-04 05:00:00+00:00,53.05,54.520,52.73,54.20,27565757.0,208706.0,53.718035,2023-01-04
2,MU,2023-01-05 05:00:00+00:00,53.89,55.390,53.89,54.71,21227741.0,153295.0,54.692799,2023-01-05
3,MU,2023-01-06 05:00:00+00:00,55.43,57.290,54.03,56.77,23910501.0,173995.0,56.297094,2023-01-06
4,MU,2023-01-09 05:00:00+00:00,56.82,57.515,55.65,56.36,17380586.0,166496.0,56.625176,2023-01-09


In [7]:
news_df = data_loader.get_news_data(
    "MU",
    start="2023-01-01",
    end="2025-06-30",
)

2025-07-09 23:58:20.018 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 1 with 50 news items
2025-07-09 23:58:20.902 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 2 with 50 news items
2025-07-09 23:58:21.779 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 3 with 50 news items
2025-07-09 23:58:22.703 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 4 with 50 news items
2025-07-09 23:58:23.567 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 5 with 50 news items
2025-07-09 23:58:24.730 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 6 with 50 news items
2025-07-09 23:58:25.600 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:364 - Fetched page 7 with 50 news items
2025-07-09 23:58:26.674 | INFO     | quantrl_lab.data.loaders.alpaca_loader:get_news_data:

In [8]:
data_processor = DataProcessor(olhcv_data=olhcv_df, news_data=news_df)

# configuring the technical indicators with specific parameters
processed_data = data_processor.data_processing_pipeline(
    indicators=["SMA", "EMA", "RSI", "MACD", "ATR", "BB", "STOCH", "OBV"],
    # The following are optional, you could do
    # data_processor.append_technical_indicators(olhcv_df, indicators=["SMA", "EMA"])
    # and the default parameters will be used.
    
    # fillna_strategy="neutral",
    SMA_params={
        "window": 20,
    },
    EMA_params={"window": 9},
    RSI_params={"window": 7},
    MACD_params={"fast": 12, "slow": 26, "signal": 9},
    ATR_params={"window": 14},
    BB_params={"window": 20, "num_std": 2},
    STOCH_params={"k_window": 14, "d_window": 3, "smooth_k": 1},
    OBV_params={},
)

2025-07-09 23:58:53.128 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 - Applying SMA with params: {'window': 20}
2025-07-09 23:58:53.132 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 - Applying EMA with params: {'window': 9}
2025-07-09 23:58:53.134 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 - Applying RSI with params: {'window': 7}
2025-07-09 23:58:53.136 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 - Applying MACD with params: {'fast': 12, 'slow': 26, 'signal': 9}
2025-07-09 23:58:53.140 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 - Applying ATR with params: {'window': 14}
2025-07-09 23:58:53.142 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 - Applying BB with params: {'window': 20, 'num_std': 2}
2025-07-09 23:58:53.146 | INFO     | quantrl_lab.data.data_processor:append_technical_indicators:226 

In [9]:
processed_data.head()

,Symbol,Timestamp,Open,High,Low,Close,Volume,Trade_count,VWAP,Date,SMA_20,EMA_9,RSI_7,MACD_line_12_26,MACD_signal_9,MACD_histogram,ATR_14,BB_middle_20,BB_upper_20_2,BB_lower_20_2,BB_bandwidth_20,STOCH_%K_14,STOCH_%D_3,OBV,sentiment_score
0,MU,2023-01-03 05:00:00+00:00,50.56,50.960,49.55,50.37,14357184.0,126221.0,50.323799,2023-01-03,NaN,50.370000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14357184.0,0.945365
1,MU,2023-01-04 05:00:00+00:00,53.05,54.520,52.73,54.20,27565757.0,208706.0,53.718035,2023-01-04,NaN,51.136000,NaN,0.305527,0.061105,0.244422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41922941.0,0.000000
2,MU,2023-01-05 05:00:00+00:00,53.89,55.390,53.89,54.71,21227741.0,153295.0,54.692799,2023-01-05,NaN,51.850800,NaN,0.582102,0.165305,0.416797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63150682.0,0.000000
3,MU,2023-01-06 05:00:00+00:00,55.43,57.290,54.03,56.77,23910501.0,173995.0,56.297094,2023-01-06,NaN,52.834640,NaN,0.956489,0.323542,0.632947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87061183.0,0.892321
4,MU,2023-01-09 05:00:00+00:00,56.82,57.515,55.65,56.36,17380586.0,166496.0,56.625176,2023-01-09,NaN,53.539712,NaN,1.206205,0.500074,0.706131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69680597.0,0.927845
